In [1]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests as req
import re

In [2]:
# assign mars news site html to variable
mars_news_html = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = req.get(mars_news_html)

# create Beautiful Soup object
mars_soup = bs(response.text, "html.parser")

In [3]:
# find mars news title 
news_title = mars_soup.find('div', class_='content_title').text

# remove \n from string
news_title = news_title.strip('\n')

news_title

"Mount Sharp 'Photobombs' Mars Curiosity Rover"

In [4]:
# find mars news paragraph 
news_p = mars_soup.find('div', class_='rollover_description_inner').text

# remove \n from string
news_p = news_p.strip('\n')

news_p

"A new self-portrait of NASA's Curiosity Mars rover shows the vehicle on Vera Rubin Ridge."

In [5]:
# assign jpl url to variable
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# Retrieve page with the requests module
response = req.get(jpl_url)

# create Beautiful Soup object
jpl_soup = bs(response.text, "html.parser")

In [6]:
# find div with carosel_items class
featured_image_url = jpl_soup.find('a', class_='fancybox')

# reassign to style attribute of div
featured_image_url = featured_image_url['data-fancybox-href']

# add jpl website prefix
featured_image_url = 'https://www.jpl.nasa.gov' + featured_image_url

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18848_ip.jpg'

In [7]:
# assign mars twitter url to variable
mars_twitter = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response = req.get(mars_twitter)

# create Beautiful Soup object
mars_tweet_soup = bs(response.text, "html.parser")

In [8]:
# find all tweets
mars_tweets = mars_tweet_soup.find_all('div', class_='tweet')

# starting timestamp for loop
latest_timestamp = 0

for tweet in mars_tweets:
    
    # collect timestamp
    tweet_timestamp = int(tweet.find('span', class_= '_timestamp')['data-time'])
    
    # collect tweet text
    tweet_text = tweet.find('p', class_ = 'tweet-text').text
    
    # assing to weather variable if tweet is mars weather and has the latest timestamp
    if (re.match(r'Sol \d\d\d\d \(', tweet_text) is not None) and (tweet_timestamp > latest_timestamp):
        
        mars_weather = tweet_text
        
        latest_timestamp = tweet_timestamp
        
mars_weather

'Sol 1951 (Jan 31, 2018), Sunny, high -23C/-9F, low -77C/-106F, pressure at 7.49 hPa, daylight 05:42-17:28'

In [9]:
# assign url to variable
mars_facts_url = 'https://space-facts.com/mars/'

# read html tables into dataframes list
mars_facts_tables = pd.read_html(mars_facts_url)

# assign mars facts table to variable
mars_facts_df = mars_facts_tables[0]

mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [10]:
# convert mars_facts_df to html string
mars_facts_html = mars_facts_df.to_html(index= False, header= None)

mars_facts_html

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [11]:
# assign hemispheres url to variable
mars_hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Retrieve page with the requests module
response = req.get(mars_hem_url)

# create mars_hem soup
mars_hem_soup = bs(response.text, 'html.parser')

In [12]:
# define browser
browser = Browser('chrome', executable_path='chromedriver', headless=False)

# visit url
browser.visit(mars_hem_url)

In [13]:
# find all hem links
mars_hem_img_list = mars_hem_soup.find_all('a', class_='itemLink')

# create list to hold title and img_url dicts
hemisphere_image_urls = []

# loop through links
for hem_img in mars_hem_img_list:
    
    hem_dict = {}
    
    # retrieve from header title
    title = hem_img.h3.text
    
    # click on title
    browser.click_link_by_partial_text(title)
    
    # find fullsize image
    full_image_url = browser.find_by_css('.wide-image').first['src']
    
    # add title and image_url to dict
    hem_dict['title'] = title
    hem_dict['img_url'] = full_image_url
    
    # add to hemisphere image list
    hemisphere_image_urls.append(hem_dict) 
    
    # go back to previous page
    browser.back()
    

hemisphere_image_urls

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]